In [ ]:
from init_notebook import *
from src.train.experiment import load_experiment_trainer

In [ ]:
trainer = load_experiment_trainer("../experiments/nexttoken//fefe2.yml", device="cpu")
trainer.model.eval()

In [ ]:
def generate(input_text: Union[str, torch.LongTensor]):
    tokenizer = trainer._tokenizer
    if isinstance(input_text, str):
        token_ids = tokenizer(input_text, return_tensors="pt").input_ids[0]
    else:
        token_ids = input_text
    if token_ids.shape[0] < trainer._seq_length:
        token_ids = torch.cat([
            torch.ones((trainer._seq_length - token_ids.shape[0],), dtype=token_ids.dtype) * tokenizer.pad_token_id,
            token_ids,
        ])
    
    print(trainer.decode(token_ids).replace("[PAD]", ""), end="")
    for i in range(500):
        logits = trainer.model(token_ids.unsqueeze(0))[0]
        best_ids = logits.argsort()[-2:]
        next_id = random.choice(best_ids).unsqueeze(0)
        token_ids = torch.cat([token_ids[1:], next_id])
        print(trainer.decode(next_id), end="")

generate(
    #"!!1!",
    #"Die Erklärung lautet: ",
    "Faschisten",
    #"Cinderella hat ein U-Boot gesprengt"
)

In [ ]:
fefe_tokens, _ = next(iter(trainer.data_loader.dataset))

In [ ]:
generate(fefe_tokens)

In [ ]:
for i, (tokens, next_token) in enumerate(trainer.data_loader.dataset):
    if i % 100 == 0:
        print(f"---- {i} -----")
        predicted = trainer.model(tokens.unsqueeze(0)).argmax(-1)
        print(f"{trainer.decode(tokens)}:[{trainer.decode(next_token)}]/[{trainer.decode(predicted)}]")
    if i > 100*10:
        break